<a href="https://colab.research.google.com/github/Swayyum/Sales-Forecasting-ML/blob/main/Sales_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 59.9 MB/s eta 0:00:00


In [6]:
from os.path import exists
import pandas as pd

df_path = "train.sv.zip" if exists("traiin.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_00, random_state=0)
df["store"]= df["store"].astype(str)
df["item"]= df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace = True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales
0,2013-01-01,5,24,26
1,2013-01-02,2,7,47
2,2013-01-03,6,21,23
3,2013-01-03,6,1,11
4,2013-01-04,9,18,60


In [9]:
train = df[df["date"]<"2017-01-01"]
test = df[df["date"]>="2017-01-01"]


In [10]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features

In [11]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
              train_target,
              eval_set=[(test_features, test_target)])

<IPython.core.display.Javascript object>

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=cd6a4ec6-44fe-4e34-a028-f02254becfc8
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



7 relevant feature(s) found with the search keys: ['date']


,,item,0.403386,100.000000,categorical,
,,store,0.138213,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_umap_35_5ddaa0ba,0.060223,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.020816,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.016188,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_24_2e14c9a6,0.010138,100.000000,numerical,Free
Upgini,Public data,f_financial_date_dow_jones_fe02128f,0.008774,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_0_1ff14cbf,0.007895,100.000000,numerical,Free
Upgini,Public data,f_economic_date_cbpol_pca_5_c87ba63a,0.000919,100.000000,numerical,Free


Calculating accuracy uplift after enrichment...


Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift,"Uplift, %"
,,,,,
Train,1518,365.963926,347.456389,18.507537,5.06
Eval 1,382,700.966346,528.326185,172.640161,24.63
